In [24]:
import requests
import pprint
import numpy as np
import time
import pickle

login = {
    'username': 'dimensionsAPI@URMC.Rochester.edu',
    'password': 'Rochester&/()API'
}
resp = requests.post('https://app.dimensions.ai/api/auth.json', json=login)
resp.raise_for_status()
headers = {
    'Authorization': "JWT " + resp.json()['token']
}

In [21]:
schools=["University of Rochester","University of California Los Angeles","Cornell University",\
         "Vanderbilt University","University of California, Berkeley","Washington University in St. Louis",\
         "Brown University","Duke University", "Northwestern University","University of Notre Dame",\
         "Columbia University","University of Pennsylvania","University of Chicago",\
         "Massachusetts Institute of Technology","California Institute of Technology","Yale University",\
         "Harvard University","Princeton University","Stanford University","Johns Hopkins University","Dartmouth College",\
         "Rice University","Emory University","Georgetown University","University of Southern California",\
        "Carnegie Mellon University","Tufts University","University of Michigan","Wake Forest University",\
        "New York University","University of California, Santa Barbara","University of North Carolina at Chapel Hill",\
        "University of California, Irvine","Brandeis University","Georgia Institute of Technology","University of Florida",\
        "Boston College","University of California, Davis","University of California, San Diego","Boston University",\
        "Case Western Reserve University","Northeastern University","Tulane University","University of Georgia",\
        "University of Illinois at Urbana Champaign","The University of Texas at Austin","University of Wisconsin",\
        "Lehigh University","Syracuse University","University of Miami","The Ohio State University","Purdue University",\
        "Rutgers University","Pennsylvania State University","Southern Methodist University","University of Washington",\
        "George Washington University","University of Connecticut","University of Maryland, College Park","University of Minnesota",\
        "University at Buffalo, State University of New York"]
with open("schools",'wb') as fp:
    pickle.dump(schools,fp)

In [25]:
with open ('menu', 'rb') as fp:
    menu = pickle.load(fp)

##### Collecting Data of Paprs 

In [22]:
whole={}
for i in schools:
    print("I'm starting for "+i)
    whole[i]=[]
# 1990-2004 2005-2011 2012-2015 2016-2019 
    for m in ["1990 and year<=2004","2005 and year<=2011","2012 and year<=2015","2016 and year<=2019"]:
        query='search publications where year>='+m+'and FOR is not empty and date is not empty and \
            research_orgs.name in ["'+i+'"] return publications[FOR+date] '
            
        resp = requests.post('https://app.dimensions.ai/api/dsl.json', 
                    data=query,headers=headers)
        j=resp.json()
        inte=j["_stats"]["total_count"]//1000
        print(inte)
        ii=0
        while True:
            try:
                for a in range(ii,inte+1):
                    resp = requests.post('https://app.dimensions.ai/api/dsl.json', 
                    data=query+'limit 1000 skip '+str(a*1000),headers=headers)
                    j=resp.json()
                    whole[i]+=j["publications"]
                    if a==inte:
                        print("Done!")
                        break
                break
            except ValueError:
                    print("try me agian")
                    time.sleep(15)
                    ii=a
                                   
    print("I'm done with "+i)
with open('paper dates for top60 schools','wb') as fp:
    pickle.dump(whole,fp)

I'm starting for University of Wisconsin
49
Done!
38
Done!
29
Done!
30
Done!
I'm done with University of Wisconsin
I'm starting for Lehigh University
4
Done!
3
Done!
2
Done!
2
Done!
I'm done with Lehigh University
I'm starting for Syracuse University
6
Done!
4
Done!
3
Done!
3
Done!
I'm done with Syracuse University
I'm starting for University of Miami
13
Done!
12
Done!
9
Done!
10
Done!
I'm done with University of Miami
I'm starting for The Ohio State University
33
Done!
28
Done!
23
Done!
24
Done!
I'm done with The Ohio State University
I'm starting for Purdue University
37
Done!
33
Done!
25
Done!
27
Done!
I'm done with Purdue University
I'm starting for Rutgers University
31
Done!
24
Done!
17
Done!
18
Done!
I'm done with Rutgers University
I'm starting for Pennsylvania State University
30
Done!
26
Done!
19
Done!
20
Done!
I'm done with Pennsylvania State University
I'm starting for Southern Methodist University
2
Done!
2
Done!
1
Done!
2
Done!
I'm done with Southern Methodist University


##### Raw data processing

In [4]:
def tocounts(dates):
    if dates!=[]:
        result={}
        for date in dates:
            if int(date[5:7])<=3:
                if date[:4]+"Q1" not in result.keys():
                    result[date[:4]+"Q1"]=0
                    result[date[:4]+"Q1"]+=1
                else:
                    result[date[:4]+"Q1"]+=1
            elif 4<=int(date[5:7])<=6:
                if date[:4]+"Q2" not in result.keys():
                    result[date[:4]+"Q2"]=0
                    result[date[:4]+"Q2"]+=1
                else:
                    result[date[:4]+"Q2"]+=1
            elif 7<=int(date[5:7])<=9:
                if date[:4]+"Q3" not in result.keys():
                    result[date[:4]+"Q3"]=0
                    result[date[:4]+"Q3"]+=1
                else:
                    result[date[:4]+"Q3"]+=1
            else:
                if date[:4]+"Q4" not in result.keys():
                    result[date[:4]+"Q4"]=0
                    result[date[:4]+"Q4"]+=1
                else:
                    result[date[:4]+"Q4"]+=1
        for i in range(1990,2020):
            for j in ["Q1","Q2","Q3","Q4"]:
                if str(i)+str(j) not in result.keys():
                    result[str(i)+str(j)]=0
        counts=[]
        for m in sorted(result):
            counts.append(result[m])
    else:
        return [0]*120
    return counts

In [5]:
with open ('forbig', 'rb') as fp:
    forbig = pickle.load(fp)
with open ('forsmall', 'rb') as fp:
    forsmall = pickle.load(fp)
with open ('paper dates for top60 schools', 'rb') as fp:
    whole = pickle.load(fp)

In [6]:
heading=[]
for i in range(1990,2020):
    for j in ["Q1","Q2","Q3","Q4"]:
        heading.append(str(i)+"-"+j)

In [7]:
import pandas
schoolsdict={}
for i in schools:
    oneschool={}
    for j in whole[i]:
        for f in j["FOR"]:
            if str(f["name"][:4]) not in oneschool.keys():
                oneschool[str(f["name"][:4])]=[]
                oneschool[str(f["name"][:4])].append(j)
            else:
                oneschool[str(f["name"][:4])].append(j)                  

    for j in oneschool.keys():
        dates=[]
        for c in oneschool[str(j)]:
            dates.append(c["date"])
        oneschool[str(j)]=tocounts(dates)
        
    vectoradd=lambda a,b:[x+y for x,y in zip(a,b)]
    
    for j in forsmall.keys():
        for f in forsmall[j]:
            if f not in oneschool.keys():
                oneschool[str(f)]=[0]*120
                
                
    for j in forsmall.keys():
        oneschool[str(j)]=[0]*120
        for f in forsmall[j]:
            oneschool[str(j)]=vectoradd(oneschool[str(j)],oneschool[str(f)])

    dffinal=pandas.DataFrame.from_dict(oneschool, orient='index')
    dffinal.columns=heading
    dffinal=dffinal.sort_index()
    schoolsdict[i]=dffinal
    
with open('schoolsdict','wb') as fp:
    pickle.dump(schoolsdict,fp)

##### Making four different-scaled dataframes

In [26]:
import pandas
with open ('schools', 'rb') as fp:
    schools = pickle.load(fp)
with open ('schoolsdict', 'rb') as fp:
    schoolsdict = pickle.load(fp)
    
for i in schools:
    schoolsdict[i]=schoolsdict[i].T

schoolsdict2={}
schoolsdict3={}
schoolsdict4={}
for i in dictofdf:
    for j in range(1,31):
        schoolsdict[i].loc[str(j+1989)] = schoolsdict[i][4*(j-1):4*j+1].sum()
    schoolsdict2[i]=schoolsdict[i][-30:]
    schoolsdict[i]=schoolsdict[i][:-30]
    schoolsdict3[i]=schoolsdict[i][["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22"]]
    schoolsdict4[i]=schoolsdict2[i][["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22"]]

with open('schoolsdict2','wb') as fp:
    pickle.dump(schoolsdict2,fp)
with open('schoolsdict3','wb') as fp:
    pickle.dump(schoolsdict3,fp)
with open('schoolsdict4','wb') as fp:
    pickle.dump(schoolsdict4,fp)

###### Definition of two Functions: correlation and correlation2

In [27]:
from scipy.stats import spearmanr
import seaborn as sns
from pandas import *

def styling(Pvalue,p,spearman,header,coefficient,menu=menu,FOR=1,school=None):
    if len(str(FOR))==1:
        FOR='0'+str(FOR)
    correlation=pandas.DataFrame.from_dict(spearman, orient='index')
    correlation.columns=header
    correlation=correlation.T
    if Pvalue:
        cmap=sns.diverging_palette(250, 5, as_cmap=True)
    else:
        cmap=sns.diverging_palette(5, 250, as_cmap=True)
        
    if Pvalue:
        def purple(val):
            border= '4px solid slateblue' if val < p else ''
            return 'border: %s' % border
    else:
        def purple(val):
            border= '4px solid slateblue' if val > coefficient else ''
            return 'border: %s' % border
    def black(s):
        if s.name=="no lag or ahead":
            return ['border:3px solid black' for v in s]
        else:
            return ['border:None' for v in s]
        
    th_props = [
      ('font-size', '5px')]
    td_props = [
      ('font-size', '5px')]
    styles = [
      dict(selector="th", props=th_props),
      dict(selector="td", props=td_props)]
    if school:
        return correlation.style.set_precision(2).apply(black,axis=1).applymap(purple).set_table_styles(styles).set_caption('U of R vs '+school).background_gradient(cmap=cmap)
    else:
        return correlation.style.apply(black,axis=1).applymap(purple).set_table_styles(styles).set_precision(2).set_caption('U of R vs other universities in categoty of'+menu[str(FOR)]).background_gradient(cmap=cmap)
def correlation(school="Duke University",dictofdf=schoolsdict2,advance=4,lag=9,Pvalue=True,coefficient=0.6,p=0.05):
    spearman={}
    header=[]
    for i in list(dictofdf["University of Rochester"].columns):
        temp=[]
        for j in range(advance,0,-1):
            if Pvalue:
                temp.append(spearmanr(dictofdf["University of Rochester"][str(i)].iloc[:j*(-1)],dictofdf[school][str(i)].iloc[j:])[1])
            else:
                temp.append(spearmanr(dictofdf["University of Rochester"][str(i)].iloc[:j*(-1)],dictofdf[school][str(i)].iloc[j:])[0])
            if len(header)<advance:header.append(str(j)+" terms ahead")
        
        if Pvalue:
            temp.append(spearmanr(dictofdf["University of Rochester"][str(i)],dictofdf[school][str(i)])[1])
        else:
            temp.append(spearmanr(dictofdf["University of Rochester"][str(i)],dictofdf[school][str(i)])[0])
        if len(header)<advance+1:header.append("no lag or ahead")
        
        for j in range(1,lag+1):
            if Pvalue:
                temp.append(spearmanr(dictofdf["University of Rochester"][str(i)].iloc[j:],dictofdf[school][str(i)].iloc[:j*(-1)])[1])
            else:
                temp.append(spearmanr(dictofdf["University of Rochester"][str(i)].iloc[j:],dictofdf[school][str(i)].iloc[:j*(-1)])[0])
            if len(header)<advance+lag+1:header.append(str(j)+" terms lag")
        spearman[i]=temp
    
    return styling(Pvalue=Pvalue,p=p,spearman=spearman,header=header,school=school,coefficient=coefficient)
def correlation2(FOR=1,schools=schools,dictofdf=schoolsdict2,lag=20,advance=7,menu=menu,Pvalue=True,coefficient=0.6,p=0.05):
    spearman={}
    header=[]
    if len(str(FOR))==1:
        FOR='0'+str(FOR)
    for i in schools:
        temp=[]
        for j in range(advance,0,-1):
            if Pvalue:
                temp.append(spearmanr(dictofdf["University of Rochester"][str(FOR)][:j*(-1)],dictofdf[i][str(FOR)][j:])[1])
            else:
                temp.append(spearmanr(dictofdf["University of Rochester"][str(FOR)][:j*(-1)],dictofdf[i][str(FOR)][j:])[0])
            if len(header)<advance:header.append(str(j)+" terms ahead")

        if Pvalue:
            temp.append(spearmanr(dictofdf["University of Rochester"][str(FOR)],dictofdf[i][str(FOR)])[1])
        else:
            temp.append(spearmanr(dictofdf["University of Rochester"][str(FOR)],dictofdf[i][str(FOR)])[0])
        if len(header)<advance+1:header.append("no lag or ahead")
        
        for j in range(1,lag+1):
            if Pvalue:
                temp.append(spearmanr(dictofdf["University of Rochester"][str(FOR)][j:],dictofdf[i][str(FOR)][:j*(-1)])[1])
            else:
                temp.append(spearmanr(dictofdf["University of Rochester"][str(FOR)][j:],dictofdf[i][str(FOR)][:j*(-1)])[0])
            if len(header)<advance+lag+1:header.append(str(j)+" terms lag")
        spearman[i]=temp

    return styling(Pvalue=Pvalue,p=p,spearman=spearman,header=header,FOR=FOR,coefficient=coefficient)

##### Different-scaled dataframes

In [28]:
schoolsdict["University of Rochester"].head()

,01,0101,0102,0103,0104,0105,0199,02,0201,0202,0203,0204,0205,0206,0299,03,0301,0302,0303,0304,0305,0306,0307,0399,04,0401,0402,0403,0404,0405,0406,0499,05,0501,0502,0503,0599,06,0601,0602,0603,0604,0605,0606,0607,0608,0699,07,0701,0702,0703,0704,0705,0706,0707,0799,08,0801,0802,0803,0804,0805,0806,0807,0899,09,0901,0902,0903,0904,0905,0906,0907,0908,0909,0910,0911,0912,0913,0914,0915,0999,10,1001,1002,1003,1004,1005,1006,1007,1099,11,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1199,12,1201,1202,1203,1204,1205,1299,13,1301,1302,1303,1399,14,1401,1402,1403,1499,15,1501,1502,1503,1504,1505,1506,1507,1599,16,1601,1602,1603,1604,1605,1606,1607,1608,1699,17,1701,1702,1799,18,1801,1802,1899,19,1901,1902,1903,1904,1905,1999,20,2001,2002,2003,2004,2005,2099,21,2101,2102,2103,2199,22,2201,2202,2203,2204,2299
1990-Q1,13,7,1,0,4,1,0,18,1,4,0,0,3,2,8,12,0,0,0,0,1,10,0,1,3,0,0,3,0,0,0,0,1,0,1,0,0,42,25,1,0,14,1,0,0,1,0,1,0,0,1,0,0,0,0,0,10,10,0,0,0,0,0,0,0,15,0,0,4,0,0,3,1,0,0,0,1,4,0,0,2,0,1,0,0,0,0,1,0,0,0,113,0,10,26,0,0,0,11,1,29,0,1,6,3,4,3,3,16,0,0,0,0,0,0,0,0,0,0,0,0,0,8,2,5,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,13,11,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,1,0,0,1,0,0,0,0,0,0,0
1990-Q2,23,7,6,0,9,1,0,56,4,20,0,0,5,1,26,18,2,1,3,0,0,9,3,0,5,0,0,4,0,0,1,0,1,0,1,0,0,65,39,1,0,23,0,0,1,1,0,0,0,0,0,0,0,0,0,0,34,19,9,1,0,0,5,0,0,20,0,0,1,0,2,2,0,0,0,0,0,11,0,1,3,0,5,0,0,0,0,5,0,0,0,168,1,19,49,0,1,0,7,5,41,2,0,7,2,7,3,8,16,0,0,0,0,0,0,0,0,2,0,0,2,0,8,2,4,2,0,1,0,1,0,0,0,0,0,0,5,0,0,0,0,2,0,1,1,1,22,21,1,0,1,1,0,0,1,0,1,0,0,0,0,5,0,2,0,3,0,0,0,0,0,0,0,1,0,0,1,0,0
1990-Q3,8,2,1,0,5,0,0,12,0,2,0,0,4,1,5,4,1,1,0,0,1,1,0,0,2,0,1,1,0,0,0,0,1,0,1,0,0,30,16,0,1,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,10,0,0,0,1,0,0,0,0,0,0,0,4,0,2,3,0,0,0,0,0,0,0,0,0,0,82,1,7,26,0,1,0,2,2,19,0,0,5,1,5,1,1,11,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,9,1,0,4,0,4,0,0,0,0,0,0,4,0,0,0,0,1,1,0,2,0,7,7,0,0,1,1,0,0,1,0,1,0,0,0,0,2,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0
1990-Q4,2,2,0,0,0,0,0,27,0,11,0,0,2,2,12,13,1,0,3,0,0,9,0,0,4,0,2,2,0,0,0,0,0,0,0,0,0,28,18,1,0,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,3,2,0,0,0,0,0,0,11,0,0,0,0,1,0,3,0,0,0,0,4,0,0,3,0,0,0,0,0,0,0,0,0,0,95,0,8,34,0,1,0,9,2,18,1,0,5,1,4,2,1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,4,0,0,1,1,0,0,0,0,0,0,0,2,0,0,0,0,1,1,0,0,0,6,6,0,0,1,1,0,0,0,0,0,0,0,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1991-Q1,11,7,0,0,4,0,0,22,1,6,0,0,3,3,9,15,1,0,1,0,1,12,0,0,2,0,0,1,0,0,1,0,0,0,0,0,0,44,32,0,0,10,1,1,0,0,0,0,0,0,0,0,0,0,0,0,7,5,1,1,0,0,0,0,0,11,0,0,0,2,0,1,0,0,0,0,0,6,0,0,2,0,2,0,0,0,0,2,0,0,0,93,0,14,23,0,1,0,6,5,18,1,1,6,1,5,5,2,5,0,0,0,0,0,0,0,0,0,0,0,0,0,7,1,5,1,0,2,0,2,0,0,0,0,0,0,3,0,0,0,0,1,0,0,1,1,7,7,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
schoolsdict2["University of Rochester"].head()

,01,0101,0102,0103,0104,0105,0199,02,0201,0202,0203,0204,0205,0206,0299,03,0301,0302,0303,0304,0305,0306,0307,0399,04,0401,0402,0403,0404,0405,0406,0499,05,0501,0502,0503,0599,06,0601,0602,0603,0604,0605,0606,0607,0608,0699,07,0701,0702,0703,0704,0705,0706,0707,0799,08,0801,0802,0803,0804,0805,0806,0807,0899,09,0901,0902,0903,0904,0905,0906,0907,0908,0909,0910,0911,0912,0913,0914,0915,0999,10,1001,1002,1003,1004,1005,1006,1007,1099,11,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1199,12,1201,1202,1203,1204,1205,1299,13,1301,1302,1303,1399,14,1401,1402,1403,1499,15,1501,1502,1503,1504,1505,1506,1507,1599,16,1601,1602,1603,1604,1605,1606,1607,1608,1699,17,1701,1702,1799,18,1801,1802,1899,19,1901,1902,1903,1904,1905,1999,20,2001,2002,2003,2004,2005,2099,21,2101,2102,2103,2199,22,2201,2202,2203,2204,2299
1990,57,25,8,0,22,2,0,135,6,43,0,0,17,9,60,62,5,2,7,0,3,41,3,1,16,0,3,11,0,0,2,0,3,0,3,0,0,209,130,3,1,68,3,1,1,2,0,1,0,0,1,0,0,0,0,0,57,37,12,2,0,0,6,0,0,67,0,0,5,3,3,6,4,0,0,0,1,29,0,3,13,0,8,0,0,0,0,8,0,0,0,551,2,58,158,0,4,0,35,15,125,4,2,29,8,25,14,15,57,0,0,0,0,0,0,0,0,2,0,0,2,0,38,6,27,5,0,9,1,8,0,0,0,0,0,0,15,0,0,0,0,6,2,1,4,2,55,52,3,0,3,3,0,0,3,0,3,0,0,0,0,11,0,4,0,7,0,0,2,0,0,2,0,1,0,0,1,0,0
1991,50,24,2,2,19,3,0,111,6,35,0,0,12,6,52,68,5,6,2,0,6,46,1,2,14,0,2,11,0,0,1,0,2,1,1,0,0,227,147,1,0,70,5,2,2,0,0,0,0,0,0,0,0,0,0,0,56,31,12,6,1,0,6,0,0,54,0,0,5,5,4,3,0,0,0,0,2,27,0,0,8,0,12,0,0,0,0,11,0,1,0,549,1,47,151,0,10,2,63,28,114,2,2,30,10,11,14,15,48,1,0,0,0,0,0,0,0,4,0,1,3,0,26,2,20,4,0,7,0,5,2,0,0,0,0,0,12,1,0,0,1,3,5,0,1,1,75,71,4,0,4,4,0,0,1,0,1,0,0,0,0,11,0,3,0,5,3,0,2,0,0,2,0,3,0,1,2,0,0
1992,45,15,5,2,22,1,0,107,3,37,0,0,6,5,56,57,6,8,3,0,3,33,3,1,22,0,3,15,0,1,3,0,3,1,2,0,0,227,130,3,0,84,7,1,2,0,0,1,0,1,0,0,0,0,0,0,49,26,2,11,3,0,7,0,0,68,0,0,6,4,4,4,3,0,1,2,2,30,1,1,10,0,14,0,0,0,1,12,0,1,0,545,1,50,161,0,11,1,37,24,78,5,2,37,19,25,19,15,57,3,0,0,0,0,0,0,0,8,0,3,5,0,49,2,40,7,0,10,0,6,4,0,0,0,0,0,22,1,0,0,1,4,11,0,4,1,74,72,2,0,3,3,0,0,0,0,0,0,0,0,0,9,0,2,0,3,4,0,6,1,0,5,0,2,1,1,0,0,0
1993,41,13,6,5,16,1,0,131,7,39,0,2,12,6,65,55,2,6,7,0,3,34,2,1,20,0,0,17,0,1,2,0,1,0,1,0,0,224,132,1,0,83,4,3,1,0,0,1,0,0,0,0,0,0,1,0,58,37,4,2,2,0,13,0,0,72,0,0,11,2,2,10,1,0,2,3,3,27,0,3,8,0,14,0,0,0,4,9,0,1,0,584,1,55,159,0,7,1,38,27,88,10,2,40,10,28,14,13,89,2,0,0,0,0,0,0,0,10,0,2,8,0,43,2,34,7,0,8,0,6,2,0,0,0,0,0,13,0,0,0,0,1,7,1,3,1,70,68,2,0,0,0,0,0,0,0,0,0,0,0,0,9,0,2,0,6,1,0,6,1,0,5,0,4,0,0,3,1,0
1994,44,12,7,6,14,5,0,142,7,53,0,3,14,4,61,86,0,7,14,0,4,54,5,2,9,0,1,7,0,0,1,0,0,0,0,0,0,246,152,0,1,88,3,1,1,0,0,2,0,0,0,0,0,0,2,0,78,45,12,9,2,0,10,0,0,76,0,0,11,3,2,4,2,1,1,3,3,38,0,0,8,0,19,0,0,0,5,14,0,0,0,731,1,93,210,0,10,0,38,34,125,9,0,46,9,27,19,9,96,5,0,0,0,0,0,0,0,16,0,2,14,0,41,1,33,7,0,8,1,5,2,0,0,0,0,0,17,1,1,0,0,2,6,1,5,1,81,78,3,0,0,0,0,0,0,0,0,0,0,0,0,9,0,1,1,6,1,0,9,0,0,9,0,2,0,0,2,0,0


In [30]:
schoolsdict3["University of California Los Angeles"].head()

,01,02,03,04,05,06,07,08,09,10,11,12,13,14,15,16,17,18,19,20,21,22
1990-Q1,20,34,32,17,3,80,2,12,35,1,256,0,2,9,1,9,42,4,0,5,3,1
1990-Q2,44,92,49,13,4,116,0,60,76,27,322,0,3,22,5,20,83,7,1,11,8,1
1990-Q3,10,23,20,9,1,90,0,13,32,5,219,0,3,13,4,10,21,0,0,2,3,0
1990-Q4,24,46,18,7,1,89,0,12,30,4,242,0,2,6,1,11,28,1,0,3,5,0
1991-Q1,20,35,24,12,2,88,0,17,39,3,296,4,2,13,5,14,43,2,1,4,8,2


In [31]:
schoolsdict4["University of Rochester"].head()

,01,02,03,04,05,06,07,08,09,10,11,12,13,14,15,16,17,18,19,20,21,22
1990,57,135,62,16,3,209,1,57,67,8,551,0,2,38,9,15,55,3,3,11,2,1
1991,50,111,68,14,2,227,0,56,54,12,549,0,4,26,7,12,75,4,1,11,2,3
1992,45,107,57,22,3,227,1,49,68,14,545,0,8,49,10,22,74,3,0,9,6,2
1993,41,131,55,20,1,224,1,58,72,14,584,0,10,43,8,13,70,0,0,9,6,4
1994,44,142,86,9,0,246,2,78,76,19,731,0,16,41,8,17,81,0,0,9,9,2


##### Correlation results

In [32]:
correlation(school="Duke University",lag=10,advance=10,
            Pvalue=False,dictofdf=dictofdf4,coefficient=0.9,p=0.05)

,01,02,03,04,05,06,07,08,09,10,11,12,13,14,15,16,17,18,19,20,21,22
10 terms ahead,0.61,0.69,0.7,0.71,0.24,0.78,-0.38,0.75,0.89,0.66,0.91,-0.15,0.7,-0.0045,0.26,0.72,0.86,0.72,-0.13,0.66,0.37,0.6
9 terms ahead,0.67,0.79,0.66,0.62,0.36,0.81,-0.24,0.8,0.9,0.69,0.92,-0.15,0.7,-0.12,0.097,0.75,0.89,0.66,0.0069,0.73,0.53,0.63
8 terms ahead,0.6,0.76,0.73,0.55,0.41,0.85,-0.088,0.84,0.88,0.71,0.93,-0.16,0.76,-0.094,-0.019,0.79,0.9,0.72,0.1,0.75,0.42,0.64
7 terms ahead,0.69,0.82,0.79,0.52,0.45,0.89,0.14,0.88,0.86,0.8,0.94,-0.15,0.77,0.05,0.2,0.81,0.91,0.76,0.25,0.81,0.52,0.71
6 terms ahead,0.74,0.83,0.83,0.55,0.4,0.89,-0.014,0.88,0.88,0.78,0.95,0.23,0.8,0.079,0.28,0.87,0.91,0.68,0.31,0.73,0.51,0.62
5 terms ahead,0.87,0.87,0.84,0.55,0.51,0.93,0.068,0.88,0.88,0.85,0.96,-0.14,0.88,0.16,0.33,0.86,0.93,0.73,0.34,0.64,0.48,0.67
4 terms ahead,0.83,0.84,0.84,0.66,0.56,0.91,0.15,0.86,0.86,0.85,0.96,-0.13,0.8,0.21,0.24,0.87,0.94,0.79,0.25,0.67,0.29,0.55
3 terms ahead,0.76,0.9,0.82,0.69,0.56,0.91,0.072,0.87,0.88,0.84,0.97,-0.13,0.78,0.16,0.43,0.86,0.94,0.88,0.27,0.8,0.51,0.63
2 terms ahead,0.8,0.9,0.86,0.71,0.6,0.9,0.14,0.88,0.91,0.81,0.97,-0.13,0.74,0.028,0.41,0.88,0.96,0.74,0.6,0.82,0.52,0.68
1 terms ahead,0.79,0.9,0.9,0.81,0.57,0.87,0.35,0.91,0.92,0.77,0.97,-0.18,0.83,0.074,0.37,0.85,0.95,0.71,0.37,0.82,0.5,0.58


In [33]:
correlation(school="Brown University",lag=10,advance=10,
            Pvalue=False,dictofdf=dictofdf4,coefficient=0.9,p=0.05)

/Users/Pro/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/Pro/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/Pro/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/Pro/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/Pro/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
/Users/Pro/anaconda3/lib/python3.6/site-packages/matplotlib/colors.py:504: RuntimeWarning: invalid value encountered in less
  xa[xa < 0]

,01,02,03,04,05,06,07,08,09,10,11,12,13,14,15,16,17,18,19,20,21,22
10 terms ahead,0.77,0.84,0.53,0.54,-0.024,0.8,0.089,0.7,0.88,0.4,0.9,-0.11,0.66,-0.11,0.015,0.73,0.87,0.52,0.25,0.66,0.33,0.72
9 terms ahead,0.87,0.87,0.58,0.62,0.12,0.82,0.076,0.7,0.85,0.54,0.93,-0.11,0.75,0.016,0.1,0.71,0.88,0.5,0.18,0.62,0.4,0.6
8 terms ahead,0.75,0.89,0.63,0.72,0.26,0.83,-0.024,0.63,0.84,0.5,0.93,-0.1,0.85,-0.21,-0.075,0.72,0.9,0.74,-0.026,0.4,0.6,0.38
7 terms ahead,0.74,0.9,0.76,0.71,0.43,0.86,0.085,0.59,0.91,0.57,0.92,-0.097,0.84,0.087,-0.041,0.75,0.91,0.82,0.11,0.57,0.52,0.58
6 terms ahead,0.78,0.91,0.74,0.72,0.49,0.87,-0.15,0.6,0.88,0.57,0.95,-0.093,0.8,0.15,0.45,0.81,0.92,0.62,0.33,0.71,0.59,0.63
5 terms ahead,0.8,0.94,0.79,0.7,0.55,0.91,0.067,0.64,0.87,0.67,0.96,-0.089,0.84,0.31,0.13,0.83,0.92,0.77,0.37,0.77,0.45,0.58
4 terms ahead,0.79,0.93,0.76,0.66,0.61,0.91,0.21,0.71,0.89,0.83,0.95,-0.085,0.81,0.25,-0.02,0.87,0.93,0.7,0.38,0.66,0.49,0.55
3 terms ahead,0.8,0.92,0.81,0.77,0.6,0.95,0.24,0.74,0.85,0.82,0.97,-0.082,0.72,0.15,0.051,0.83,0.94,0.8,0.22,0.68,0.37,0.52
2 terms ahead,0.85,0.89,0.81,0.77,0.62,0.93,0.27,0.76,0.84,0.76,0.97,-0.078,0.7,-0.11,0.46,0.84,0.95,0.76,0.45,0.76,0.55,0.43
1 terms ahead,0.82,0.89,0.82,0.77,0.61,0.88,0.59,0.8,0.87,0.75,0.97,-0.11,0.74,-0.055,0.34,0.88,0.95,0.6,0.5,0.77,0.39,0.45


In [34]:
correlation2(FOR=1,lag=12,advance=12,Pvalue=False,dictofdf=schoolsdict2,coefficient=0.85,p=0.05)